In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com/data"

In [ ]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv(aws_s3_dir + '/boundaries/v_0/boundary_georef.csv')
boundary_georef

In [ ]:
# # remove cities with time out issues due to volumetry
# data_volumetry_error_cities = ['MEX-Mexico_City']
# boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(data_volumetry_error_cities)].reset_index(drop=True)
# boundary_georef

# Compute indicator

In [ ]:
cities_indicators_GRE_3_1 = pd.DataFrame() 

In [ ]:
for i in range(7,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    #read open space
    openspace_path = aws_s3_dir + '/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

    ## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.
    WCprojection = esa_land_cover.projection(); 
    RecSitesImg = openspace_geo_ee.style(
      color='gray',
    ).reproject(
          crs= WCprojection
        )
    # create image with two bands: BuiltupPixels and UrbanOpenPixels
    Builtup = esa_land_cover.updateMask(esa_land_cover.eq(50)).rename("BuiltupPixels")
    UrbanOpen = RecSitesImg.updateMask(esa_land_cover.eq(50)).select(1).rename("UrbanOpenPixels")
    comb = Builtup.addBands([UrbanOpen])
    
    # Function to translate pixel counts into area and percents
    def CountToArea(feat):
        feat = ee.Feature(feat)
        FeatArea = ee.Number(feat.area(1)).multiply(0.000001)
        UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))

        return feat.set({
            #'TotalareaKM2': FeatArea,
            #'UrbanOpenAreaKM2': UrbanOpenArea,
            #'BuiltupAreaKM2': BuiltupArea,
            'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea,
        })
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## define scale for reductions - 10 is ideal, but can be increased if memory errors with large geographies
    Scale = 10
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= Scale, 
      tileScale= 1
    )

    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= Scale, 
      tileScale= 1
    )
    
    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)

In [ ]:
cities_indicators_GRE_3_1

# Workaround for geographies with a lot of open space that won't load into memory - must be run one city at a time

In [ ]:
#delete any existing GEE asset for openspace
ee.data.deleteAsset('users/emackres/thisopenspace')

In [ ]:
## calculate city - adjust range to include single city of interest
for i in range(6,7): #len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    #read open space
    openspace_path = aws_s3_dir + '/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

    exportTask = ee.batch.Export.table.toAsset(
        collection = openspace_geo_ee,
        description = 'description',
        assetId = 'users/emackres/thisopenspace'
    )
    exportTask.start()

In [ ]:
exportTask.status()

In [ ]:
## load saved asset - wait until task status says "COMPLETED"
openspace_geo_ee = ee.FeatureCollection('users/emackres/thisopenspace')

In [ ]:
for i in range(6,7): #len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
        
    # #read open space
    # openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    # openspace_geo = requests.get(openspace_path).json()
    # openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

    ## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.
    WCprojection = esa_land_cover.projection(); 
    RecSitesImg = openspace_geo_ee.style(
      color='gray',
    ).reproject(
          crs= WCprojection
        )
    # create image with two bands: BuiltupPixels and UrbanOpenPixels
    Builtup = esa_land_cover.updateMask(esa_land_cover.eq(50)).rename("BuiltupPixels")
    UrbanOpen = RecSitesImg.updateMask(esa_land_cover.eq(50)).select(1).rename("UrbanOpenPixels")
    comb = Builtup.addBands([UrbanOpen])
    
    # Function to translate pixel counts into area and percents
    def CountToArea(feat):
        feat = ee.Feature(feat)
        FeatArea = ee.Number(feat.area(1)).multiply(0.000001)
        UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))

        return feat.set({
            #'TotalareaKM2': FeatArea,
            #'UrbanOpenAreaKM2': UrbanOpenArea,
            #'BuiltupAreaKM2': BuiltupArea,
            'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea,
        })
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## define scale for reductions - 10 is ideal, but can be increased if memory errors with large geographies
    Scale = 10
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= Scale, 
      tileScale= 1
    )

    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= Scale, 
      tileScale= 1
    )
    
    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)

In [ ]:
cities_indicators_GRE_3_1

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv('https://'+bucket_name+'.s3.eu-west-3.amazonaws.com/indicators/cities_indicators_ericV1.csv')
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_GRE_3_1,
                                            indicator_name = "GRE_3_1_percentOpenSpaceinBuiltup")

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'indicators/cities_indicators_ericV1.csv'

cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')